In [8]:
import pandas as pd
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn. metrics import roc_auc_score, accuracy_score
import seaborn as sns

In [9]:
df = pd.read_csv('../data/train_1M_with_features_merged_v2.csv')

In [10]:
df.head()

,Unnamed: 0,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,...,user_avg_score_cum_on_this_part,user_correct_answers_cum_on_this_part,part1,part2,part3,part4,part5,part6,part7,already_seen_tag_q_l_cumcount_with_trim_at_3
0,0,90286654,0,1917429896,7900,0,0,1,NaN,0,...,0.0,0.0,1,0,0,0,0,0,0,0
1,1,90286655,26511,1917429896,7876,0,1,0,21000.0,0,...,1.0,1.0,1,0,0,0,0,0,0,0
2,2,90286656,55312,1917429896,175,0,2,0,21000.0,0,...,0.5,1.0,1,0,0,0,0,0,0,0
3,3,90286657,102436,1917429896,1278,0,3,0,23000.0,0,...,0.0,0.0,0,1,0,0,0,0,0,0
4,4,90286658,188454,1917429896,2063,0,4,0,45000.0,0,...,0.0,0.0,0,0,1,0,0,0,0,0


In [11]:
from sklearn.preprocessing import RobustScaler
corr = df.query('content_type_id==0').corr().answered_correctly
corr.sort_values(ascending=False)

answered_correctly                                                 1.000000
user_personalized_qstat_knowing_had_explanation_or_not             0.393328
qstats_answered_correctly                                          0.380770
qstats_answered_correctly_knowing_having_had_explanation           0.373850
qstats_answered_correctly_knowing_having_not_had_explanation       0.211393
user_avg_score_cum_on_this_part                                    0.189773
user_avg_score_cum                                                 0.181107
qstats_prior_question_had_explanation                              0.109755
prior_question_had_explanation                                     0.100927
already_seen_tag_q_l_cumcount_with_trim_at_3                       0.097745
already_seen_tag_lecture                                           0.086766
already_seen                                                       0.073562
user_activity_cumcount                                             0.073021
task_contain

# Essai xgboost 14 features

In [12]:
#corr[corr > 0.04].sort_values(ascending=False).index.to_list()
features = ['user_personalized_qstat_knowing_had_explanation_or_not',
            'user_avg_score_cum_on_this_part',
            'user_correct_answers_cum_on_this_part',
 'qstats_answered_correctly',
 'user_avg_score_cum',
 'qstats_prior_question_had_explanation',
 'already_seen',
 #'already_seen_tag_lecture',
 'user_activity_cumcount',
 #'qstats_user_activity_cumcount',
 'at_least_one_lesson',
 #'qstats_user_activity_cumcount_knowing_answered_correctly',
 #'qstats_task_container_id_knowing_answered_correctly',
 #'lectures_seen',
 #'qstats_count',
 #'qstats_answers',
 'content_type_id']

In [13]:
feats=df.loc[:,features]
feats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 987269 entries, 0 to 987268
Data columns (total 10 columns):
 #   Column                                                  Non-Null Count   Dtype  
---  ------                                                  --------------   -----  
 0   user_personalized_qstat_knowing_had_explanation_or_not  987269 non-null  float64
 1   user_avg_score_cum_on_this_part                         967834 non-null  float64
 2   user_correct_answers_cum_on_this_part                   967834 non-null  float64
 3   qstats_answered_correctly                               967834 non-null  float64
 4   user_avg_score_cum                                      967834 non-null  float64
 5   qstats_prior_question_had_explanation                   967834 non-null  float64
 6   already_seen                                            987269 non-null  int64  
 7   user_activity_cumcount                                  987269 non-null  int64  
 8   at_least_one_lesson     

In [14]:
import numpy as np
feats = feats.replace(np.inf, 0.5)
feats

,user_personalized_qstat_knowing_had_explanation_or_not,user_avg_score_cum_on_this_part,user_correct_answers_cum_on_this_part,qstats_answered_correctly,user_avg_score_cum,qstats_prior_question_had_explanation,already_seen,user_activity_cumcount,at_least_one_lesson,content_type_id
0,0.8364,0.0,0.0,0.8370,0.000000,0.07404,0,0,0,0
1,0.3770,1.0,1.0,0.3972,1.000000,0.11970,0,1,0,0
2,0.3513,0.5,1.0,0.3640,0.500000,0.14160,0,2,0,0
3,0.6450,0.0,0.0,0.6514,0.333333,0.05120,0,3,0,0
4,0.1642,0.0,0.0,0.1749,0.250000,0.09250,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...
987264,0.5930,1.0,3.0,0.5967,0.600000,0.23200,0,25,0,0
987265,0.4854,0.0,0.0,0.5063,0.615385,0.07830,0,26,0,0
987266,0.4270,1.0,1.0,0.4424,0.629630,0.07830,0,27,0,0
987267,0.2842,1.0,2.0,0.2932,0.642857,0.07830,0,28,0,0


In [9]:
feats.replace(np.nan, 0, inplace=True)

Best so far - XGBoost


In [27]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from skearn

X = feats.query('content_type_id==0').drop(columns={'content_type_id'})
y = df.query('content_type_id==0')['answered_correctly']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 2)

scaler = MinMaxScaler()
xgboost = HistGradientBoostingClassifier()
xgboost_pipe = make_pipeline(scaler,xgboost)


y_pred = cross_val_predict(xgboost_pipe, X, y, cv=5, method='predict_proba', verbose=1, njobs=-1)
print('roc-auc:', roc_auc_score(y, y_pred[:,1]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/thomasmathieu/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/thomasmathieu/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/thomasmathieu/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/thomasmathieu/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Dat

roc-auc: 0.7668300129201188


In [43]:
xgboost_pipe

TypeError: 'Pipeline' object is not subscriptable

In [ ]:
features.columns

Index(['user_personalized_qstat_knowing_had_explanation_or_not',
       'qstats_answered_correctly', 'user_avg_score_cum',
       'qstats_prior_question_had_explanation', 'already_seen',
       'already_seen_tag_lecture', 'user_activity_cumcount',
       'qstats_user_activity_cumcount', 'at_least_one_lesson',
       'qstats_user_activity_cumcount_knowing_answered_correctly',
       'qstats_task_container_id_knowing_answered_correctly', 'lectures_seen',
       'qstats_count', 'qstats_answers', 'content_type_id'],
      dtype='object')

# En enlevant la cross val

In [15]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

X = feats.query('content_type_id==0').drop(columns={'content_type_id'})
y = df.query('content_type_id==0')['answered_correctly']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 1)

scaler = MinMaxScaler()
xgboost = HistGradientBoostingClassifier()
xgboost_pipe = make_pipeline(scaler,xgboost)

xgboost_pipe.fit (X_train, y_train)

y_pred = xgboost_pipe.predict_proba(X_test)
print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

roc-auc: 0.7717767085029563


In [89]:
xgboost.feature_importances_

array([0.74222709, 0.06279859, 0.00116907, 0.00843532, 0.09884094,
       0.0053311 , 0.05941764, 0.01913869, 0.00264155])

# Re-run sur tout le train et export pickle

In [16]:
xgboost_pipe.fit (X, y)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier())])

In [17]:
import pickle
pickle.dump(xgboost_pipe, open("xgboost_pipe3.pkl", "wb") )

In [92]:
features = ['user_personalized_qstat_knowing_had_explanation_or_not',
            'user_avg_score_cum_on_this_part',
            'user_correct_answers_cum_on_this_part',
 'qstats_answered_correctly',
 'user_avg_score_cum',
 'qstats_prior_question_had_explanation',
 'already_seen',
 #'already_seen_tag_lecture',
 'user_activity_cumcount',
 #'qstats_user_activity_cumcount',
 'at_least_one_lesson',
 #'qstats_user_activity_cumcount_knowing_answered_correctly',
 #'qstats_task_container_id_knowing_answered_correctly',
 #'lectures_seen',
 #'qstats_count',
 #'qstats_answers',
 'content_type_id']

pd.Series(features[:-1]).to_csv('xgboost_pipe3_features_list.csv')

pd.Series(features[:-1])

0    user_personalized_qstat_knowing_had_explanatio...
1                      user_avg_score_cum_on_this_part
2                user_correct_answers_cum_on_this_part
3                            qstats_answered_correctly
4                                   user_avg_score_cum
5                qstats_prior_question_had_explanation
6                                         already_seen
7                               user_activity_cumcount
8                                  at_least_one_lesson
dtype: object

# Logistic Regression

In [ ]:
X = df.query('content_type_id==0')[['qstats_answered_correctly', 'user_avg_score_cum', 'already_seen', 'lectures_seen', 'at_least_one_lesson']]
y = df.query('content_type_id==0')['answered_correctly']

In [ ]:
X = features.query('content_type_id==0').drop(columns={'content_type_id'})
y = df.query('content_type_id==0')['answered_correctly']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 2)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

log = LogisticRegression()
log.fit(X_train, y_train)
y_pred = cross_val_predict(log, X_test, y_test, cv=10, method='predict_proba', verbose=1)
print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

Correlations

In [ ]:
corr.sort_values(ascending=False)

In [ ]:
corr = df.query('content_type_id==0').corr().answered_correctly

# RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X = df.query('content_type_id==0')[['qstats_answered_correctly', 'user_avg_score_cum', 'already_seen', 'lectures_seen', 'at_least_one_lesson']]
y = df.query('content_type_id==0')['answered_correctly']

In [ ]:
y_pred = cross_val_predict(forest, X_test, y_test, cv=10, method='predict_proba', verbose=1)
print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


roc-auc: 0.7125921037505767


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  7.0min finished


# Bagging Classifier

# Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 1)
adaboost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), n_estimators=50) 
adaboost.fit(X_train, y_train)

y_pred = cross_val_predict(adaboost, X_test, y_test, cv=10, method='predict_proba', verbose=1)
print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


roc-auc: 0.7594259462782705


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  2.3min finished


# SVC

In [ ]:
from sklearn.svm import SVC

X = features.query('content_type_id==0').drop(columns={'content_type_id'})
y = df.query('content_type_id==0')['answered_correctly']

kernels = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']

for kernel in kernels:
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 2)
  X_train = scaler.fit_transform(X_train)
  svc = SVC(kernel=kernel, C=10, max_iter=1500, probability=True, tol=0.01, gamma='scale')
  svc.fit(X_train, y_train)
  y_pred = cross_val_predict(svc, X_test, y_test, cv=5, method='predict_proba', verbose=1)
  print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

In [ ]:
# equivalent but with SGD solver
from sklearn.linear_model import SGDClassifier

svc_bis = SGDClassifier(loss='modified_huber', penalty='l2', alpha=0.1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = 1)

#svc_bis = GradientBoostingClassifier() 
svc_bis.fit(X_train, y_train)

y_pred = cross_val_predict(svc_bis, X_test, y_test, cv=10, method='predict_proba', verbose=1)
print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


roc-auc: 0.7289342611288083


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    4.3s finished


# Multi-layer Stacking - XGBoost, SVC and Log reg

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()

X = features.query('content_type_id==0').drop(columns={'content_type_id'})
y = df.query('content_type_id==0')['answered_correctly']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 2)

X_train = scaler.fit_transform(X_train)


ensemble = StackingClassifier(
    estimators = [("log", GradientBoostingClassifier()),
                  ("svc", SGDClassifier(loss='modified_huber', penalty='l2', alpha=0.1, max_iter=3000))],
                  final_estimator = LogisticRegression(max_iter=3000) )

ensemble.fit(X_train, y_train)
y_pred = cross_val_predict(ensemble, X_test, y_test, cv=5, method='predict_proba', verbose=1)
print('roc-auc:', roc_auc_score(y_test, y_pred[:,1]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


roc-auc: 0.7682322465652832


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 25.1min finished


In [6]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

In [2]:
pip freeze

absl-py==0.11.0
aiobotocore==1.1.2
aiohttp==3.7.3
aioitertools==0.7.1
alembic==1.4.1
altair==4.1.0
appnope==0.1.0
argon2-cffi==20.1.0
astor==0.8.1
astroid==2.4.2
astunparse==1.6.3
async-generator==1.10
async-timeout==3.0.1
attrs==20.2.0
azure-core==1.9.0
azure-storage-blob==12.6.0
backcall==0.2.0
base58==2.0.1
beautifulsoup4==4.9.3
bleach==3.2.1
blinker==1.4
botocore==1.17.44
branca==0.4.1
bs4==0.0.1
cachetools==4.1.1
category-encoders==2.2.2
certifi==2020.6.20
cffi==1.14.3
chardet==3.0.4
chromedriver-binary==86.0.4240.22.0
click==7.1.2
cloudpickle==1.6.0
confuse==1.3.0
coverage==5.3
cryptography==3.2.1
cycler==0.10.0
Cython==0.29.17
databricks-cli==0.14.1
deap==1.3.1
decorator==4.4.2
defusedxml==0.6.0
docker==4.4.0
docutils==0.15.2
entrypoints==0.3
Flask==1.1.1
Flask-Cors==3.0.9
folium==0.11.0
fsspec==0.8.4
future==0.18.2
gast==0.3.3
gcsfs==0.6.0
gensim==3.8.3
gitdb==4.0.5
GitPython==3.1.11
google-api-core==1.23.0
google-auth==1.23.0
google-auth-oauthlib==0.4.2
google-cloud-core==1.4.